# Introducción a GeoPandas

> _Preparado por: **Juan Javier Santos Ochoa** ([LNPP](https://www.lnpp.mx/))_. 
> Contacto: [web](https://www.jjsantoso.com/), [LinkedIn](https://www.linkedin.com/in/jsantosochoa/), [Twitter](https://twitter.com/jjsantoso)

<img src="imagenes/geopandas.jpg" style="width: 800px;"/>

[GeoPandas](http://geopandas.org/index.html) es una libreria de Python que hace muy fácil trabajar con datos geoespaciales. 
GeoPandas extiende las funciones de la popular librería [Pandas](https://pandas.pydata.org/) para permitir hacer operaciones con datos geométricos. Para ello, usa algunas librerías que ya existen en Python: para hacer operaciones geométricas como [shapely](http://toblerity.github.io/shapely), para la lectura de archivos usa [Fiona](http://toblerity.org/fiona/README.html) y para graficar usa [Matplotlib](https://matplotlib.org/).

**¿Para qué usar GeoPandas?**
* Para análisis exploratorio de datos geoespaciales
* Para hacer operaciones vectoriales de objetos geoespaciales
* Hacer visualizaciones sencillas de datos goegráficos


## Instalación

Para esta sesión usaremos la versión 0.5.0 de GeoPandas. Asumo que tiene instalada la distribución [Anaconda](https://www.anaconda.com/download/).

Para instalarlo vamos a la terminal de comandos de la computadora y escribimos:

```terminal
conda install geopandas
```

Les mostrará una lista de paquetes que se instalarán, también los paquetes que serán actualizados y los que serán rebajados. Les pedirá que acepten presionando `y`+`enter`.

## Importando la librería

In [ ]:
import geopandas as gpd
import sys
print(gpd.__name__, gpd.__version__)
print(sys.version)
%matplotlib inline

> `%matplotlib inline` es una magic function para ver las gráficas que generamos dentro del notebook

# Estructuras de datos

GeoPandas tiene dos principales estructuras de datos que son los `GeoDataFrames` y `GeoSeries`, que son subclases de los `DataFrames` y `Series` de Pandas.

## GeoSerie

Una GeoSerie es un vector donde cada entrada es un conjunto de formas geométricas (_shapes_) correspondiente a una observación.

Una entrada puede consistir de solo una forma (como el polígono para un estado) o de varias formas que son una sola observación (como un conjunto de islas que tiene varios polígonos).

GeoPandas tiene tres tipos básicas de formas geométricas:
* **Puntos/ Multipuntos**: Un punto es un conjunto de exactamente una coordenada. Una colección de puntos es un objeto Multipunto.

    * Un punto lucirá en una GeoSerie como un obejto `Point(x,y)` donde `x` y `y` son las coordenadas. 
    
* **Lineas/ Multilineas**: Una línea se definirá a partir de la unión de varios puntos definidos en una lista. El orden viene dado por la posición en la lista. 

    * En una GeoSerie una línea se verá como un objeto: `LineString([(x1, y1), (x2, y2)])`
  
* **Polígonos/ Multipolígonos**: Un polígono igualmente es una colleción de puntos en una lista, como una línea, con la diferencia de que debe ser cerrado. Implícitamente asume que el último punto de la lista se une con el primero. Un polígono también puede tener "huecos".

    * En una GeoSerie un polígono se vera como un objeto: `Polygon([(0, 0), (1, 1), (1, 0)])`
    
## GeoDataFrame

Es una estructura tabular, como un _DataFrame_ de Pandas, que contiene una _GeoSerie_. La característica más importante de un _GeoDataFrame_ es que tiene una columna _GeoSerie_ que tiene un caracter especial. Esta columna es referida como la "geometría" (_geometry_) del `gdf`.

Cuando se aplica un método espacial a un `gdf`, este se aplica sobre la _geometry_

Se puede acceder a la columna "_geometry_", la variable puede llamarse diferente, a través del atributo `.geometry` ( gdf.geometry), y se puede encontrar el nombre de la columna que es la _geometry_ escribiendo `gdf.geometry.name`

Un `gdf` puede tener varias _GeoSeries_ pero solo una será la `geometry` activa a la vez. Para establecer la columna que es la `geometry` se usa el método `.set_geometry()`

## Ejemplo

Importaremos un conjunto de datos que viene incluído en GeoPandas. Este contiene los países del mundo y algunos datos básicos como la población y el continente al que pertenecen.

En este `gdf` la geometry es una GeoSerie que se llama "geometry"

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world

In [ ]:
# Comprobamos es un GeoDataFrame
type(world)

In [ ]:
# comprobamos el nombre de la geometry
print(world.geometry.name)

In [ ]:
# Comprobamos que es una GeoSerie
type(world['geometry'])

In [ ]:
# Veamos cómo se ve!!
print('Hola mundo!',"\U0001F632")
world.plot()

# Leyendo archivos espaciales

GeoPandas puede abrir varios tipos de archivos de formato geoespacial, sin embargo, en esta sesión usaremos _shapefiles_ (.shp), que son un tipo de archivo muy popular. Otros tipos de archivos populares es `GeoJson`.

Usaremos un _shape_ de todos los municipios de México. El mapa de México proveniente del Marco Geoestadístico Nacional de INEGI lo pueden encontrar [aquí](https://www.inegi.org.mx/temas/mg/default.html#Descargas).




## Desde un shapefile

In [ ]:
## Leemos el archivo
mx = gpd.read_file('datos/mapa_mexico/Division_Municipal_Mexico_2010.shp')
mx.head()

In [ ]:
mx.plot()

In [ ]:
# Establecemos el índice
mx.set_index('CLAVE', inplace=True)
# Revisamos el GDF
mx.loc[['09004', '09003']].plot()

## Sistema de Coordenadas Geográficas

Un [Sistema de coordenadas geográficas](http://desktop.arcgis.com/es/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm) (CRS) define un marco para identificar ubicaciones en un globo 3D. Estos se componen de varios parámetros que definen la posición de las coordenadas con respecto a la tierra. 

Existen muchos CRS y es muy importante que cuando trabajemos con datos geográficos estas coordenadas estén definidas, y de preferencia que las sistemas de coordinados de todas nuestras bases de datos coincida. Escoger un CRS depende de lo que se quiera hacer y de la escala, entre otros aspectos.

Existen también los sistemas de proyección geográfica, que definen la forma como transformamos las coordenadas geográficas de una esfera/elipse a un plano 2d.

En `GeoPandas` podemos consultar el sistema de coordenadas geográficas de un `GeoDataFrame` con el atributo `.crs`

In [ ]:
print(world.crs, mx.crs)

## Puntos desde un csv

Un '.csv' no se puede importar directamente como un GeoDataFrame, pero podemos crear primero un DataFrame y crear una columna geometry usando la función `gpd.points_from_xy()`

In [ ]:
from shapely.geometry import Polygon

In [ ]:
import pandas as pd
# leemos CSV con PANDAS
df_est = pd.read_csv('datos/estaciones.csv')
# creamos GeoDataFrame a partir de DataFrame
estaciones =  gpd.GeoDataFrame(df_est, geometry=gpd.points_from_xy(df_est['longitud'], df_est['latitud']), crs='epsg:4326')
estaciones.head()

In [ ]:
# Graficamos
estaciones.plot()

## Reproyectar

Para cambiar el sistema de proyecciones de un GeoDataFrame usamos el método `.to_crs()`. Hay varias formas de especificar el nuevo sistema de coordenadas, en el siguiente ejemplo usamos el número epsg 3395 (Mercator) y luego definimos la Azimuthal Equidistant Pojection.

In [ ]:
# En windows parece haber un problema de instalación que no permite reproyectar. 
# La solución es incluir una variable de ambiente con la carpeta
# Library/share de la instalación de Python
import os
anaconda_dir = gpd.__file__.split('lib')[0]
os.environ["PROJ_LIB"] = os.path.join(anaconda_dir, 'Library/share')

In [ ]:
world.to_crs(epsg=3395).head()

In [ ]:
world.to_crs(epsg=3395).plot()

In [ ]:
world.to_crs(crs='+proj=aeqd +lat_0=60 +lon_0=0 +x_0=0').plot()

In [ ]:
mx = mx.to_crs(epsg=4485)
mx.plot()

In [ ]:
estaciones = estaciones.to_crs(epsg=4485)
estaciones.plot()

# Seleccionando datos

Podemos usar los métodos para seleccionar observaciones disponibles para un `DataFrame`, como `.iloc[]`, `.loc[]`, `.query()`.

In [ ]:
# usamos .loc para seleccionar las observaciones con los índices
mx.loc[['09002','09011']].plot()

In [ ]:
mx.query('CVE_EDO=="09"').plot()

## Tarea en clase

Muestre un mapa con todos los municipios del estado de Tlaxcala y Puebla

# Operaciones geométricas

Las `GeoSeries` y `GeoDataFrames` tienen algunos atributos y métodos para hacer operaciones geométricas básicas.

## Centroides

Es el punto medio geográfico de una entidad espacial.


In [ ]:
centroides = mx.centroid
centroides.head()

In [ ]:
centroides.plot(c='red', markersize=0.5)

## Bordes

* Obtiene los bordes de los polígonos, pero no su interior

In [ ]:
bordes = mx.boundary
bordes.head()

In [ ]:
bordes.plot(color='green', lw=0.5, figsize=(8, 10))

## Buffer
un buffer es el área que está a determinada ditancia de un punto. La unidad de medida de la distancia viene dada por la proyección geográfica.

In [ ]:
# defino un nuevo GDF que se llama cdmx
cdmx = mx.query('CVE_EDO=="09"').reset_index()
# Calculo el buffer
buff = estaciones.buffer(2000)
buff.head()

In [ ]:
# Grafico el buffer, o asigno a la variable ax
ax = buff.plot(color='blue')

In [ ]:
## CENTROIDE CON SU BUFFER y BORDES
ax = buff.plot(color='blue')
estaciones.plot(color='red', ax=ax)
cdmx.boundary.plot(color='green', ax=ax)

## Unary union

Realiza la unión de todos los elementos de un `GeoDataFrame`

In [ ]:
cdmx_union = cdmx.unary_union
cdmx_union

In [ ]:
type(cdmx_union)

## Intersección

In [ ]:
buff.unary_union

In [ ]:
## vamos a intersectar el GDF de la CDMX con el buffer que calculamos
cdmx.intersection(buff.unary_union).plot()

## Diferencia

In [ ]:
## vamos a calcular la diferencia del GDF de la CDMX con el buffer que calculamos. 
#Esto es, lo que nos quedaría si a la CDMX le quitamos el área de los buffers
diferencia = cdmx.difference(buff.unary_union)
diferencia

In [ ]:
diferencia.loc[~diferencia.is_empty].plot()

## Disolver

Realiza la unión de las entidades según el valor de alguna variable.

In [ ]:
estados = mx.dissolve(by='CVE_EDO')
estados.head()

In [ ]:
estados.plot()

## Contención

Sirve para verificar si los elementos de una `GeoSerie` contienen un punto u otra forma espacial.

In [ ]:
contiene = cdmx.contains(estaciones.geometry.iloc[0])
contiene

In [ ]:
cdmx.loc[contiene]

## Tarea en Clase

Selecciones los municipios del Estado de México y guárdelos en un GeoDataframe que se llame edomex

* Grafique los bordes de los municipios de la CDMX y los del estado de México. Use un color distinto para cada estado.
* Grafique los centroides de los municipios de la CDMX y los del estado de México. Use un color distinto para cada estado.

# Guardando información

Los GeoDataFrames que generamos los podemos guardar como shapefiles:

In [ ]:
estados.to_file('datos/estados')

# Mapas Coropléticos

Podemos hacer algunos mapas básicos con GeoPandas.

In [ ]:
estados_pib = gpd.read_file('datos/estados_pib/')
estados_pib.head()

In [ ]:
# El color representa el valor de la variable PIB_pc
ax = estados_pib.plot(column='pib_pc', cmap='OrRd', scheme='Quantiles', legend=True,
                           linewidth=1.5, edgecolor='k', k=4)
ax.set_title('PIB per cápita, estatal, 2016')
ax.axis('off')
leyenda = ax.get_legend()
leyenda.set_bbox_to_anchor((.12, .4))
leyenda.set_title('Intervalos')
# Guardamos la figura
ax.figure.savefig('graficas/mapa_estados.png', dpi=600, bbox_inches='tight')

# Otras librerías
Hay varias opciones para análizar y visualizar información geográfica en Python. Algunas recomendadas son:

* Gráficas estáticas:
    * [GeoPlot](https://residentmario.github.io/geoplot/index.html): Gráficas estáticas basadas en GeoPandas.
    * [CartoPy](https://scitools.org.uk/cartopy/docs/latest/): Gráficas estáticas basadas en matplotlib.
    * [Datashader](http://datashader.org/): permite graficar millones de datos en poco tiempo.
* Gráficas dinámicas
    * [Geoviews](http://geoviews.org/): Genera gráficas dinámicas usando como backend diferentes librerías (Bokeh, matplotlib).
    * [Folium](http://python-visualization.github.io/folium/): Genera mapas interactivos dentro de Jupyter Notebooks.
    * [Bokeh](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html): es una librería de visualización de gráficas interactivas con gran soporte para información geográfica. También se pueden hacer dashboards.
    * [Ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet): Genera mapas interactivos dentro de Jupyter Notebooks y se puede integrar con otros widgets.
    
    

* Análisis Espacial y software GIS
    * [Pysal](https://pysal.readthedocs.io/en/latest/): Librería especializada en análisis y modelos espaciales. También puede graficar.
    * [OSMnx](https://github.com/gboeing/osmnx): Librería que permite descargar y visualizar datos de Open Street Map.
    * [Pyqgis](https://docs.qgis.org/testing/en/docs/pyqgis_developer_cookbook/): Es la librería de Python que utiliza el conocido software GIS [Qgis](https://qgis.org/es/site/) de acceso libre.
    * [cartoframes](https://carto.com/developers/cartoframes/examples/): Es la librería de Python creada por la compañia especializada en inteligencia espacial [Carto](https://carto.com/) para acceder a sus servicios.
    * [ArcGis API](https://developers.arcgis.com/python/guide/overview-of-the-arcgis-api-for-python/): Es la librería de Python que utiliza el conocido software  [ArcGis](https://www.arcgis.com/home/index.html) de acceso con pago. Algunos de sus notebooks se pueden ejecutar en línea: [ejemplo](https://notebooks.esri.com/user/ttNgOgaHvpIgJfNFDvEfZdL9A/notebooks/samples/04_gis_analysts_data_scientists/finding_hospitals_closest_to_an_incident.ipynb)
    
# Referencias

* [Documentación de Geopandas](https://geopandas.org/)
* [Curso Automating GIS processes](https://automating-gis-processes.github.io/site/)
